In [1]:
import warnings
import argparse
import DrawingWithTensors
import pandas as pd
from random import randint
import torch
import os
import argparse
import time

# initiate the parser
parser = argparse.ArgumentParser(description = "List of options to run application when creating custom datset")

# options for parser
parser = argparse.ArgumentParser()  
parser.add_argument("-V", "--version", help="show program version", action="store_true")
parser.add_argument("-s", "--size", help="upper bound of number of images to include")
parser.add_argument("-t", "--rotations", help="allow random image and tensor rotations to be made")
parser.add_argument("-f", "--root_folder", help="destination for root folder")
parser.add_argument("-g", "--generation", help="which generation number we are using")
parser.add_argument("-d", "--dimensions", help="square image dimensions")
parser.add_argument("-y", "--scale", help="feet/pixel ratio")

# defined defaults
upper_bound = 10
isize = 416
iteration = "0"
rotations= False
ROOT = os.getcwd()
factor = 0.45

# read arguments from the command line if empty, or with notebook parameters
# args = parser.parse_args()
args = parser.parse_args(['-g','11',
                          '-s','100000',
                          '-d','416',
                          '-t','true',
                          '-f', '/media/peo5032/My Passport/input/gen',
                          '-y', '0.15'
                         ])

# check for --version or -V
if args.version:  
    print("this is version 0.1", flush=True)
    
if args.size: 
    print("will create", upper_bound, "images", flush=True)
    upper_bound = int(args.size) + 1

if args.rotations: 
    rotations= args.rotations.lower() == "true"
    print("rotations was", args.rotations, flush=True)
    
if args.root_folder:  
    os.makedirs(args.root_folder, exist_ok=True)
    print("destination was", args.root_folder, flush=True)
    ROOT = args.root_folder
    
if args.generation:
    print("iteration was", args.generation, flush=True)
    iteration = args.generation
    
if args.dimensions:
    print("dimension chosen was", args.dimensions,flush=True)
    isize = int(args.dimensions)
    
if args.scale:
    print("scale ratio chosen was", args.scale,flush=True)
    factor = args.scale
    

os.makedirs(ROOT, exist_ok=True)
IMAGE_PATH = ROOT + iteration + "/roads"
os.makedirs(IMAGE_PATH, exist_ok=True)

TENSOR_PATH = ROOT + iteration + "/tensor_values"
os.makedirs(TENSOR_PATH, exist_ok=True)

PICKLE_PATH = ROOT + iteration

df = pd.DataFrame()
NumLanes = []
ShldrWidth = []
ShldrWidthCenter = []
RoadWidth = []
FileNames = []
imageGen = DrawingWithTensors.datasetFactory(IMAGE_SIZE = isize)
b_type = ["trees", "water", "desert"]

/opt/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


will create 10 images
rotations was true
destination was /media/peo5032/My Passport/input/gen
iteration was 11
dimension chosen was 416
scale ratio chosen was 0.15


In [2]:
tmp_tensor = torch.zeros(1,isize,isize,dtype=torch.float32)
since = time.time()
for i in range(0,upper_bound):
    
    if i % 10 == 0:
        print("Pic {:0d}/{:0d}".format(i,upper_bound), flush=True)
    c = randint(0,80)
    lanecount = randint(1,3)
    laneWidth = randint(17,40)
    lineWidth = randint(1,2)
    shoulderWidth = randint(0,79)
    
    #create tuple of information, img, and tensor
    tuple,img,tmp_tensor = imageGen.generateNewImageWithTensor(c,
                                                               lanecount,
                                                               laneWidth,
                                                               lineWidth,
                                                               shoulderWidth,
                                                               tmp_tensor,
                                                               b_type[randint(0,len(b_type)-1)],
                                                               factor)
    
    roadWidth,laneCount,shoulderWidth,centerShldrWidth = tuple       
    
    if rotations:
        img, tmp_tensor = DrawingWithTensors.rotationOfImageAndTensor(img,
                                                                      tmp_tensor,
                                                                      imageGen.classList,
                                                                      randint(0,180),
                                                                     )
        
    NumLanes.append(laneCount)
    ShldrWidth.append(shoulderWidth)
    RoadWidth.append (roadWidth)
    ShldrWidthCenter.append(centerShldrWidth)
    
    FileName = str(i) + ".png"
    FileNames.append(FileName)
    img.save(IMAGE_PATH + "/" + FileName,"PNG")
    
    
    torch.save(tmp_tensor, TENSOR_PATH + "/"+ str(i) + '.pt')
    
    img.close()
    tmp_tensor = torch.mul(tmp_tensor, 0)

    #reintroduce pickle file if desired
    
    #df['NumLanes'] = NumLanes
    #df['ShldrWidth'] = ShldrWidth
    #df['RdwyWidth'] = RoadWidth
    #df['ShldrWidthCenter'] = ShldrWidthCenter
    #df['FileName'] = FileNames

#df.to_pickle(PICKLE_PATH + "/train_images_v2.pkl")

time_elapsed = time.time() - since
print('Generation complete in {:.1f}m {:.1f}s'.format(time_elapsed // 60, time_elapsed % 60), flush=True)

Pic 0/100001
Pic 10/100001
Pic 20/100001
Pic 30/100001
Pic 40/100001
Pic 50/100001
Pic 60/100001
Pic 70/100001
Pic 80/100001
Pic 90/100001
Pic 100/100001
Pic 110/100001
Pic 120/100001
Pic 130/100001
Pic 140/100001
Pic 150/100001
Pic 160/100001
Pic 170/100001
Pic 180/100001
Pic 190/100001
Pic 200/100001
Pic 210/100001
Pic 220/100001
Pic 230/100001
Pic 240/100001
Pic 250/100001
Pic 260/100001
Pic 270/100001
Pic 280/100001
Pic 290/100001
Pic 300/100001
Pic 310/100001
Pic 320/100001
Pic 330/100001
Pic 340/100001
Pic 350/100001
Pic 360/100001
Pic 370/100001
Pic 380/100001
Pic 390/100001
Pic 400/100001
Pic 410/100001
Pic 420/100001
Pic 430/100001
Pic 440/100001
Pic 450/100001
Pic 460/100001
Pic 470/100001
Pic 480/100001
Pic 490/100001
Pic 500/100001
Pic 510/100001
Pic 520/100001
Pic 530/100001
Pic 540/100001
Pic 550/100001
Pic 560/100001
Pic 570/100001
Pic 580/100001
Pic 590/100001
Pic 600/100001
Pic 610/100001
Pic 620/100001
Pic 630/100001
Pic 640/100001
Pic 650/100001
Pic 660/100001
Pic 67

Pic 5190/100001
Pic 5200/100001
Pic 5210/100001
Pic 5220/100001
Pic 5230/100001
Pic 5240/100001
Pic 5250/100001
Pic 5260/100001
Pic 5270/100001
Pic 5280/100001
Pic 5290/100001
Pic 5300/100001
Pic 5310/100001
Pic 5320/100001
Pic 5330/100001
Pic 5340/100001
Pic 5350/100001
Pic 5360/100001
Pic 5370/100001
Pic 5380/100001
Pic 5390/100001
Pic 5400/100001
Pic 5410/100001
Pic 5420/100001
Pic 5430/100001
Pic 5440/100001
Pic 5450/100001
Pic 5460/100001
Pic 5470/100001
Pic 5480/100001
Pic 5490/100001
Pic 5500/100001
Pic 5510/100001
Pic 5520/100001
Pic 5530/100001
Pic 5540/100001
Pic 5550/100001
Pic 5560/100001
Pic 5570/100001
Pic 5580/100001
Pic 5590/100001
Pic 5600/100001
Pic 5610/100001
Pic 5620/100001
Pic 5630/100001
Pic 5640/100001
Pic 5650/100001
Pic 5660/100001
Pic 5670/100001
Pic 5680/100001
Pic 5690/100001
Pic 5700/100001
Pic 5710/100001
Pic 5720/100001
Pic 5730/100001
Pic 5740/100001
Pic 5750/100001
Pic 5760/100001
Pic 5770/100001
Pic 5780/100001
Pic 5790/100001
Pic 5800/100001
Pic 5810

Pic 10300/100001
Pic 10310/100001
Pic 10320/100001
Pic 10330/100001
Pic 10340/100001
Pic 10350/100001
Pic 10360/100001
Pic 10370/100001
Pic 10380/100001
Pic 10390/100001
Pic 10400/100001
Pic 10410/100001
Pic 10420/100001
Pic 10430/100001
Pic 10440/100001
Pic 10450/100001
Pic 10460/100001
Pic 10470/100001
Pic 10480/100001
Pic 10490/100001
Pic 10500/100001
Pic 10510/100001
Pic 10520/100001
Pic 10530/100001
Pic 10540/100001
Pic 10550/100001
Pic 10560/100001
Pic 10570/100001
Pic 10580/100001
Pic 10590/100001
Pic 10600/100001
Pic 10610/100001
Pic 10620/100001
Pic 10630/100001
Pic 10640/100001
Pic 10650/100001
Pic 10660/100001
Pic 10670/100001
Pic 10680/100001
Pic 10690/100001
Pic 10700/100001
Pic 10710/100001
Pic 10720/100001
Pic 10730/100001
Pic 10740/100001
Pic 10750/100001
Pic 10760/100001
Pic 10770/100001
Pic 10780/100001
Pic 10790/100001
Pic 10800/100001
Pic 10810/100001
Pic 10820/100001
Pic 10830/100001
Pic 10840/100001
Pic 10850/100001
Pic 10860/100001
Pic 10870/100001
Pic 10880/1000

Pic 15120/100001
Pic 15130/100001
Pic 15140/100001
Pic 15150/100001
Pic 15160/100001
Pic 15170/100001
Pic 15180/100001
Pic 15190/100001
Pic 15200/100001
Pic 15210/100001
Pic 15220/100001
Pic 15230/100001
Pic 15240/100001
Pic 15250/100001
Pic 15260/100001
Pic 15270/100001
Pic 15280/100001
Pic 15290/100001
Pic 15300/100001
Pic 15310/100001
Pic 15320/100001
Pic 15330/100001
Pic 15340/100001
Pic 15350/100001
Pic 15360/100001
Pic 15370/100001
Pic 15380/100001
Pic 15390/100001
Pic 15400/100001
Pic 15410/100001
Pic 15420/100001
Pic 15430/100001
Pic 15440/100001
Pic 15450/100001
Pic 15460/100001
Pic 15470/100001
Pic 15480/100001
Pic 15490/100001
Pic 15500/100001
Pic 15510/100001
Pic 15520/100001
Pic 15530/100001
Pic 15540/100001
Pic 15550/100001
Pic 15560/100001
Pic 15570/100001
Pic 15580/100001
Pic 15590/100001
Pic 15600/100001
Pic 15610/100001
Pic 15620/100001
Pic 15630/100001
Pic 15640/100001
Pic 15650/100001
Pic 15660/100001
Pic 15670/100001
Pic 15680/100001
Pic 15690/100001
Pic 15700/1000

Pic 19940/100001
Pic 19950/100001
Pic 19960/100001
Pic 19970/100001
Pic 19980/100001
Pic 19990/100001
Pic 20000/100001
Pic 20010/100001
Pic 20020/100001
Pic 20030/100001
Pic 20040/100001
Pic 20050/100001
Pic 20060/100001
Pic 20070/100001
Pic 20080/100001
Pic 20090/100001
Pic 20100/100001
Pic 20110/100001
Pic 20120/100001
Pic 20130/100001
Pic 20140/100001
Pic 20150/100001
Pic 20160/100001
Pic 20170/100001
Pic 20180/100001
Pic 20190/100001
Pic 20200/100001
Pic 20210/100001
Pic 20220/100001
Pic 20230/100001
Pic 20240/100001
Pic 20250/100001
Pic 20260/100001
Pic 20270/100001
Pic 20280/100001
Pic 20290/100001
Pic 20300/100001
Pic 20310/100001
Pic 20320/100001
Pic 20330/100001
Pic 20340/100001
Pic 20350/100001
Pic 20360/100001
Pic 20370/100001
Pic 20380/100001
Pic 20390/100001
Pic 20400/100001
Pic 20410/100001
Pic 20420/100001
Pic 20430/100001
Pic 20440/100001
Pic 20450/100001
Pic 20460/100001
Pic 20470/100001
Pic 20480/100001
Pic 20490/100001
Pic 20500/100001
Pic 20510/100001
Pic 20520/1000

...

Pic 77780/100001
Pic 77790/100001
Pic 77800/100001
Pic 77810/100001
Pic 77820/100001
Pic 77830/100001
Pic 77840/100001
Pic 77850/100001
Pic 77860/100001
Pic 77870/100001
Pic 77880/100001
Pic 77890/100001
Pic 77900/100001
Pic 77910/100001
Pic 77920/100001
Pic 77930/100001
Pic 77940/100001
Pic 77950/100001
Pic 77960/100001
Pic 77970/100001
Pic 77980/100001
Pic 77990/100001
Pic 78000/100001
Pic 78010/100001
Pic 78020/100001
Pic 78030/100001
Pic 78040/100001
Pic 78050/100001
Pic 78060/100001
Pic 78070/100001
Pic 78080/100001
Pic 78090/100001
Pic 78100/100001
Pic 78110/100001
Pic 78120/100001
Pic 78130/100001
Pic 78140/100001
Pic 78150/100001
Pic 78160/100001
Pic 78170/100001
Pic 78180/100001
Pic 78190/100001
Pic 78200/100001
Pic 78210/100001
Pic 78220/100001
Pic 78230/100001
Pic 78240/100001
Pic 78250/100001
Pic 78260/100001
Pic 78270/100001
Pic 78280/100001
Pic 78290/100001
Pic 78300/100001
Pic 78310/100001
Pic 78320/100001
Pic 78330/100001
Pic 78340/100001
Pic 78350/100001
Pic 78360/1000

Pic 82600/100001
Pic 82610/100001
Pic 82620/100001
Pic 82630/100001
Pic 82640/100001
Pic 82650/100001
Pic 82660/100001
Pic 82670/100001
Pic 82680/100001
Pic 82690/100001
Pic 82700/100001
Pic 82710/100001
Pic 82720/100001
Pic 82730/100001
Pic 82740/100001
Pic 82750/100001
Pic 82760/100001
Pic 82770/100001
Pic 82780/100001
Pic 82790/100001
Pic 82800/100001
Pic 82810/100001
Pic 82820/100001
Pic 82830/100001
Pic 82840/100001
Pic 82850/100001
Pic 82860/100001
Pic 82870/100001
Pic 82880/100001
Pic 82890/100001
Pic 82900/100001
Pic 82910/100001
Pic 82920/100001
Pic 82930/100001
Pic 82940/100001
Pic 82950/100001
Pic 82960/100001
Pic 82970/100001
Pic 82980/100001
Pic 82990/100001
Pic 83000/100001
Pic 83010/100001
Pic 83020/100001
Pic 83030/100001
Pic 83040/100001
Pic 83050/100001
Pic 83060/100001
Pic 83070/100001
Pic 83080/100001
Pic 83090/100001
Pic 83100/100001
Pic 83110/100001
Pic 83120/100001
Pic 83130/100001
Pic 83140/100001
Pic 83150/100001
Pic 83160/100001
Pic 83170/100001
Pic 83180/1000

Pic 87420/100001
Pic 87430/100001
Pic 87440/100001
Pic 87450/100001
Pic 87460/100001
Pic 87470/100001
Pic 87480/100001
Pic 87490/100001
Pic 87500/100001
Pic 87510/100001
Pic 87520/100001
Pic 87530/100001
Pic 87540/100001
Pic 87550/100001
Pic 87560/100001
Pic 87570/100001
Pic 87580/100001
Pic 87590/100001
Pic 87600/100001
Pic 87610/100001
Pic 87620/100001
Pic 87630/100001
Pic 87640/100001
Pic 87650/100001
Pic 87660/100001
Pic 87670/100001
Pic 87680/100001
Pic 87690/100001
Pic 87700/100001
Pic 87710/100001
Pic 87720/100001
Pic 87730/100001
Pic 87740/100001
Pic 87750/100001
Pic 87760/100001
Pic 87770/100001
Pic 87780/100001
Pic 87790/100001
Pic 87800/100001
Pic 87810/100001
Pic 87820/100001
Pic 87830/100001
Pic 87840/100001
Pic 87850/100001
Pic 87860/100001
Pic 87870/100001
Pic 87880/100001
Pic 87890/100001
Pic 87900/100001
Pic 87910/100001
Pic 87920/100001
Pic 87930/100001
Pic 87940/100001
Pic 87950/100001
Pic 87960/100001
Pic 87970/100001
Pic 87980/100001
Pic 87990/100001
Pic 88000/1000

Pic 92240/100001
Pic 92250/100001
Pic 92260/100001
Pic 92270/100001
Pic 92280/100001
Pic 92290/100001
Pic 92300/100001
Pic 92310/100001
Pic 92320/100001
Pic 92330/100001
Pic 92340/100001
Pic 92350/100001
Pic 92360/100001
Pic 92370/100001
Pic 92380/100001
Pic 92390/100001
Pic 92400/100001
Pic 92410/100001
Pic 92420/100001
Pic 92430/100001
Pic 92440/100001
Pic 92450/100001
Pic 92460/100001
Pic 92470/100001
Pic 92480/100001
Pic 92490/100001
Pic 92500/100001
Pic 92510/100001
Pic 92520/100001
Pic 92530/100001
Pic 92540/100001
Pic 92550/100001
Pic 92560/100001
Pic 92570/100001
Pic 92580/100001
Pic 92590/100001
Pic 92600/100001
Pic 92610/100001
Pic 92620/100001
Pic 92630/100001
Pic 92640/100001
Pic 92650/100001
Pic 92660/100001
Pic 92670/100001
Pic 92680/100001
Pic 92690/100001
Pic 92700/100001
Pic 92710/100001
Pic 92720/100001
Pic 92730/100001
Pic 92740/100001
Pic 92750/100001
Pic 92760/100001
Pic 92770/100001
Pic 92780/100001
Pic 92790/100001
Pic 92800/100001
Pic 92810/100001
Pic 92820/1000

Pic 97060/100001
Pic 97070/100001
Pic 97080/100001
Pic 97090/100001
Pic 97100/100001
Pic 97110/100001
Pic 97120/100001
Pic 97130/100001
Pic 97140/100001
Pic 97150/100001
Pic 97160/100001
Pic 97170/100001
Pic 97180/100001
Pic 97190/100001
Pic 97200/100001
Pic 97210/100001
Pic 97220/100001
Pic 97230/100001
Pic 97240/100001
Pic 97250/100001
Pic 97260/100001
Pic 97270/100001
Pic 97280/100001
Pic 97290/100001
Pic 97300/100001
Pic 97310/100001
Pic 97320/100001
Pic 97330/100001
Pic 97340/100001
Pic 97350/100001
Pic 97360/100001
Pic 97370/100001
Pic 97380/100001
Pic 97390/100001
Pic 97400/100001
Pic 97410/100001
Pic 97420/100001
Pic 97430/100001
Pic 97440/100001
Pic 97450/100001
Pic 97460/100001
Pic 97470/100001
Pic 97480/100001
Pic 97490/100001
Pic 97500/100001
Pic 97510/100001
Pic 97520/100001
Pic 97530/100001
Pic 97540/100001
Pic 97550/100001
Pic 97560/100001
Pic 97570/100001
Pic 97580/100001
Pic 97590/100001
Pic 97600/100001
Pic 97610/100001
Pic 97620/100001
Pic 97630/100001
Pic 97640/1000

In [3]:
#from PIL import Image
#testing output of data-set
#img = Image.open(IMAGE_PATH+"/11.png")
#test_tensor = torch.load(TENSOR_PATH+'/11.pt')
#img

In [4]:
#DrawingWithTensors.showInferenceOnImage(img, test_tensor, "road", 0.6, imageGen.classMap)